# Agent Development with Plan-Execute Mode and Plan Confirmation


1. Define tools
2. Define model, memory
3. Define nodes
4. Define graph, connect nodes with edges
5. Print execution plan and confirm
6. Execute the plan

安装依赖

    pip install -U langchain-openai
    pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

设置 OpenAI KEY

    export OPENAI_API_KEY="sk-xxx"

In [1]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState
from typing import List
import json

## 1、Define Tools

In [2]:
@tool
def search_companies_by_industry(industry: str) -> str:
    """Input industry name, return a company list ID. Must use this tool to get ID before using search_people_by_company_list to query people."""
    return "company_list_123"

@tool
def get_company_list_id_from_names(companies: list) -> str:
    """Input a list of company names, return a company list ID for subsequent batch people queries."""
    return "company_list_from_names_456"
    
@tool
def expand_job_title(base_title: str) -> list:
    """
    Use a large model to generate a list of related job titles from a given job title.
    Example: "Software Engineer" → ["Software Engineer", "Backend Developer", "Fullstack Engineer", "R&D Engineer"]
    """
    llm = init_chat_model("gpt-4o-mini", temperature=0)
    prompt = f"""
    You are an HR expert. Given the job title "{base_title}"，
    Please generate a JSON array with 3~5 related job titles.
    Output JSON array only, no extra text, no markdown.
    """
    resp = llm.invoke(prompt)
    try:
        print('expand_job_title:', resp.content)
        import json
        job_list = json.loads(resp.content)
        if isinstance(job_list, list):
            return job_list
    except Exception:
        raise
    return [base_title]  # fallback

@tool
def search_people_by_company_list(company_list_id: str, job_titles: list) -> list:
    """
    Input company list ID and multiple job keywords to batch search people in these companies.
    Example job_titles: ["Software Engineer", "Backend Developer", "R&D Engineer"]
    """
    results = []
    for title in job_titles:
        results.append({
            "name": f"Test-{title}",
            "title": title,
            "company": "DemoCorp"
        })
    return results    

@tool
def get_phone_number_by_name(name: str) -> str:
    """Input a person's name, return the phone number."""
    phone_book = {
        "Test-Software Developer": "13800000001",
        "Test-Backend Engineer": "13800000002",
        "Test-Full Stack Developer": "13800000003",
        "Test-R&D Engineer": "13800000004",
        "Test-DevOps Engineer": "13800000004"
    }
    return phone_book.get(name, "Unknown")    

## 2、Define model, memory

In [3]:
# ====== Create model & tools list ======
tools = [
    search_companies_by_industry,
    get_company_list_id_from_names,
    expand_job_title,
    search_people_by_company_list,
    get_phone_number_by_name
]
model = init_chat_model("gpt-4o-mini", temperature=0).bind_tools(tools)
memory = MemorySaver()


## 3、Define graph, connect nodes with edges

In [4]:
# ------- plan node -------
def plan_node(state: MessagesState):
    user_query = state["messages"][-1].content
    plan_prompt = f"""
    User requirement：{user_query}
    Please reason step by step which tools to call, what input each tool requires, and the calling order.
    Output as a JSON array, each element containing "tool", "input", and "purpose".
    Do not execute tools, only plan. Do not output markdown formatting.
    """
    resp = model.invoke([("system", "You are a task planning assistant"), ("user", plan_prompt)])
    plan_text = resp.content
    print("\n=== AI Task Plan ===\n")
    print(plan_text)
    return {"messages": state["messages"] + [resp], "plan_text": plan_text, "user_query": user_query}


# ------- execute node -------
def execute_node(state: MessagesState):
    print("\n=== Start Excute Plan ===\n")
    # Directly call ReAct agent mode
    # Reuse the same model & tools here, context is preserved
    from langgraph.prebuilt import create_react_agent
    agent = create_react_agent(model, tools, checkpointer=memory)
    config = {"configurable": {"thread_id": "thread-1"}}
       
    plan_text = state.get("plan_text", '')
    user_query = state.get("user_query", '')
    prompt = f"""
    The user's question is：{user_query}
    Please strictly follow the plan below, do not generate a new plan:
    {plan_text}
    """
    events = agent.stream({"messages": [("user", prompt)]}, config)
    
    for event in events:
        if "agent" in event:
            for msg in event["agent"]["messages"]:
                msg.pretty_print()
        if "tools" in event:
            for msg in event["tools"]["messages"]:
                msg.pretty_print()
    return state

## 4、Define graph, connect nodes with edges

In [5]:
graph = StateGraph(MessagesState)
# ====== Add nodes to graph ======
graph.add_node("plan", plan_node)
graph.add_node("execute", execute_node)

graph.add_edge("plan", "execute")
graph.add_edge("execute", END)

graph.set_entry_point("plan")
app = graph.compile(checkpointer=memory)

## 5、Print execution plan and confirm

In [6]:
# ===== Stream excute =====
user_input = "Help me find software engineers in these companies and provide phone numbers: Alibaba, Tencent"
config = {"configurable": {"thread_id": "thread-1"}}

# Run only to the plan node the first time, then stop.
events = app.stream({"messages": [("user", user_input)]}, config, stop_at=["plan"])

state_after_plan = None
for event in events:
    if "plan" in event:
        state_after_plan = app.get_state(config)
        break


=== AI Task Plan ===

[
    {
        "tool": "functions.get_company_list_id_from_names",
        "input": {
            "companies": ["Alibaba", "Tencent"]
        },
        "purpose": "To obtain a company list ID for Alibaba and Tencent, which is required for the next step to search for people."
    },
    {
        "tool": "functions.expand_job_title",
        "input": {
            "base_title": "Software Engineer"
        },
        "purpose": "To generate a list of related job titles for 'Software Engineer' that can be used to broaden the search for relevant individuals."
    },
    {
        "tool": "functions.search_people_by_company_list",
        "input": {
            "company_list_id": "<company_list_id>",  // This will be replaced with the actual ID obtained from the first step
            "job_titles": "<expanded_job_titles>"  // This will be replaced with the list of job titles obtained from the second step
        },
        "purpose": "To search for people working as

## 6、Execute the plan

In [7]:
# Confirm execution
if state_after_plan:
    # confirm = input("\nConfirm execution？(y/n)：").strip().lower()
    confirm = 'y' # for test
    if confirm == "y":
        for event in events:
            if "agent" in event:
                for msg in event["agent"]["messages"]:
                    msg.pretty_print()
            if "tools" in event:
                for msg in event["tools"]["messages"]:
                    msg.pretty_print()
    else:
        print("Execution cancelled.。")



=== Start Excute Plan ===

================================== Ai Message ==================================
Tool Calls:
  get_company_list_id_from_names (call_OrOcHu5dQpM9GeFz0cd3in0V)
 Call ID: call_OrOcHu5dQpM9GeFz0cd3in0V
  Args:
    companies: ['Alibaba', 'Tencent']
================================= Tool Message =================================
Name: get_company_list_id_from_names

company_list_from_names_456
================================== Ai Message ==================================
Tool Calls:
  expand_job_title (call_4urBFc85yQswm1abkbF4ZsXF)
 Call ID: call_4urBFc85yQswm1abkbF4ZsXF
  Args:
    base_title: Software Engineer
expand_job_title: [
    "Software Developer",
    "Full Stack Developer",
    "Backend Engineer",
    "Frontend Developer",
    "DevOps Engineer"
]
================================= Tool Message =================================
Name: expand_job_title

["Software Developer", "Full Stack Developer", "Backend Engineer", "Frontend Developer", "DevOps Engin